In [7]:
import numpy as np
from numpy.random import randn
import numpy as np
import random


In [12]:
# Taking Raw Sentiments data

train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'i am happy':True,
  'this is not good':False
}

test_data = {
  'is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is good':True
  
}


In [13]:

# Vocabulary.
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size)

# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }

def createInputs(text):

 # Returns an array of one-hot vectors representing the words in the input text string.

  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

9 unique words found


In [14]:
class RNN:
  def __init__(self, input_size, output_size, hidden_size=64):

    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  def forward(self, inputs):
  
    # Method for forward pass of the RNN 
    h = np.zeros((self.Whh.shape[0], 1))

    self.last_inputs = inputs
    self.last_hs = { 0: h }

    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by

    return y, h

  def backprop(self, d_y, learn_rate=2e-2):
   
    # Method for Backword pass of the RNN 
    n = len(self.last_inputs)

    # Calculate Diffrentiation .
    d_Why = d_y @ self.last_hs[n].T
    d_by = d_y

    # Initialize variables to zero.
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)

    # Calculate dL/dh for the last h.
    # dL/dh = dL/dy * dy/dh
    d_h = self.Why.T @ d_y

    # Backpropagate through time.
    for t in reversed(range(n)):
      temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)
      d_bh += temp
      d_Whh += temp @ self.last_hs[t].T
      d_Wxh += temp @ self.last_inputs[t].T
      d_h = self.Whh @ temp

    for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
      np.clip(d, -1, 1, out=d)

    # Update weights and biases using gradient descent.
    self.Whh -= learn_rate * d_Whh
    self.Wxh -= learn_rate * d_Wxh
    self.Why -= learn_rate * d_Why
    self.bh -= learn_rate * d_bh
    self.by -= learn_rate * d_by


In [15]:

# Initialize our RNN!
rnn = RNN(vocab_size, 2)

def processRNN(data, backprop=True):
  
  #Applying the constructed RNN on data
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x)
    target = int(y)

    # Forward
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      d_L_d_y[target] -= 1

      # Backward
      rnn.backprop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

# Training loop
for epoch in range(500):
  train_loss, train_acc = processRNN(train_data)

  if epoch % 50 == 25:
    print('-----------Epoch %d-----------' % (epoch + 1))
  

    test_loss, test_acc = processRNN(test_data, backprop=False)
    print('Test:\tLoss %.2f | Accuracy: %.2f' % (test_loss, test_acc))


-----------Epoch 26-----------
Test:	Loss 0.72 | Accuracy: 0.40
-----------Epoch 76-----------
Test:	Loss 0.72 | Accuracy: 0.40
-----------Epoch 126-----------
Test:	Loss 0.72 | Accuracy: 0.40
-----------Epoch 176-----------
Test:	Loss 0.73 | Accuracy: 0.40
-----------Epoch 226-----------
Test:	Loss 0.78 | Accuracy: 0.40
-----------Epoch 276-----------
Test:	Loss 0.83 | Accuracy: 0.40
-----------Epoch 326-----------
Test:	Loss 0.83 | Accuracy: 0.40
-----------Epoch 376-----------
Test:	Loss 0.83 | Accuracy: 0.40
-----------Epoch 426-----------
Test:	Loss 0.78 | Accuracy: 0.40
-----------Epoch 476-----------
Test:	Loss 0.31 | Accuracy: 1.00
